In [152]:
import json
from fastapi.testclient import TestClient
import importlib
import backend
importlib.reload(backend)  # 重新加载模块
from backend import app
from typing import List
import uuid
from datetime import datetime

# Initialize TestClient
client = TestClient(app)


In [147]:
product = "size changing and adjustable garment"
hazard = ""
model_steering = "There is a reported hazard in the product which is {hazard}"
number_of_stakeholders_generated = 1
number_of_stakeholders_to_use = 1

stakeholders = []
interviews = []
questions = [
    "What challenges do you face with current garments?",
    "How would an adjustable garment benefit you?",
    "What features would you expect in such a garment?",
]

In [148]:
latent_needs = []

In [150]:
def create_stakeholder_to_use():
    payload = {
        "design_context": product,
        "model_steering": model_steering,
        "number_of_stakeholders": number_of_stakeholders_generated
    }

    response = client.post("/create_stakeholder_serial", json=payload)
    #response = requests.post(f"{"http://127.0.0.1:8000"}/create_stakeholder_serial", json=payload)
    data = response.json()
    print("Created Stakeholders (Serial):", data)

    global stakeholders
    stakeholders = data["stakeholders"]

    # filter top stakeholders
    if len(stakeholders) > number_of_stakeholders_to_use:

        for s in stakeholders:
            if 'id' not in s:
                s['id'] = str(uuid.uuid4())

        payload = {
            "stakeholders": stakeholders,
            "n": number_of_stakeholders_to_use
        }

        response = client.post("/filter_top_diverse", json=payload)
        assert response.status_code == 200
        data = response.json()
        print("\nFiltered Stakeholders (Top N Diverse):", json.dumps(data, indent=2))
        stakeholders = data
    
    for stakeholder in stakeholders:
        print(stakeholder)

In [151]:
create_stakeholder_to_use()

DEBUG:asyncio:Using selector: KqueueSelector
DEBUG:backend:Creating 1 users.
DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'user', 'content': "You're a user creator for a new design of size changing and adjustable garment. You MUST consider the following: Focus on adaptive wearability. The users must be diverse consumers of the product, whom you will interview afterwards to identify user needs. Please create a list of 1 users."}], 'model': 'gpt-4o', 'function_call': {'name': 'StakeholderList'}, 'functions': [{'name': 'StakeholderList', 'description': 'Correctly extracted `StakeholderList` with all the required parameters with correct types', 'parameters': {'$defs': {'StakeholderBase': {'properties': {'chain_of_thought': {'description': '\nExplain why this stakeholder is important to the design process.\n', 'title': 'Chain Of Thought', 'type': 'string'}, 'stakeholder_type': {'description': 'Ty

Created Stakeholders (Serial): {'stakeholders': [{'chain_of_thought': "This stakeholder is critical because they are representative of a market segment that is eager to adopt and advocate for innovative fashion solutions. Their enthusiasm for adaptive clothing makes them an ideal candidate to provide insightful feedback on the garment's wearability and style. Their tech-savvy nature ensures they'll engage deeply with the garment's adjustable features. Additionally, their sensitivity to inclusivity and sustainability will guide the design toward meeting diverse consumer needs and expectations.", 'stakeholder_type': 'Adaptive Clothing Enthusiast', 'personality_description': 'The user is highly adaptive, fashion-forward, and technology-savvy. They are enthusiastic about innovations in the clothing industry, especially those that promote inclusivity and adaptability. They are conscious of sustainability and appreciate garments that offer practicality without compromising style. They value 

In [153]:
def test_conduct_interviews():
    print("\n=== Testing Interview Conduct ===")
    global interviews
    global stakeholders

    interviews = []
    
    for stakeholder in stakeholders:
        if 'id' not in stakeholder:
            stakeholder['id'] = str(uuid.uuid4())  # 需要导入 uuid
        payload = {
            "stakeholder": stakeholder,
            "design_context": product,
            "questions": questions
        }

        
        response = client.post("/conduct_interview", json=payload)
        interview_data = response.json()
        interviews.append(interview_data)
        print(f"\nInterview with {stakeholder['stakeholder_type']}:")
        print(json.dumps(interview_data, indent=2))

In [ ]:
test_conduct_interviews()

In [155]:
def identify_latent_needs():
    print("\n=== Identifying Latent Needs ===")
    global latent_needs
    latent_needs = []
    
    for interview in interviews:
        payload = {
            "design_context": product,
            "interview": interview
        }

        try:
            response = client.post("/latent_need_identification", json=payload)
            print(f"Raw response: {response.text}")
            if response.status_code == 200:
                need_data = response.json()
                # 后端返回的是 LatentNeeds 对象，包含 latent_needs 列表
                if isinstance(need_data, dict) and 'latent_needs' in need_data:
                    needs = need_data['latent_needs']
                    latent_needs.extend(needs)
                    print(f"\nLatent needs identified for {interview['stakeholder_type']}:")
                    for need in needs:
                        print("\nNeed:", need.get('need'))
                        print("Chain of Thought:", need.get('chain_of_thought'))
                        print("Is Latent:", need.get('latent'))
                else:
                    print(f"\nUnexpected response format: {need_data}")
            else:
                print(f"\nError identifying latent needs for {interview['stakeholder_type']}")
                print(f"Status code: {response.status_code}")
                print(f"Response: {response.text}")
        except Exception as e:
            print(f"\nException during latent need identification for {interview['stakeholder_type']}")
            print(f"Error: {str(e)}")
            print(f"Response content: {response.text if 'response' in locals() else 'No response'}")
                 
        # try:
        #     response = client.post("/latent_need_identification", json=payload)
        #     if response.status_code == 200:
        #         need_data = response.json()
        #         latent_needs.append(need_data)
        #         print(f"\nLatent needs identified for {interview['stakeholder_type']}:")
        #     else:
        #         print(f"\nError identifying latent needs for {interview['stakeholder_type']}")
        #         print(f"Response: {response.text}")
        # except Exception as e:
        #     print(f"\nException during latent need identification for {interview['stakeholder_type']}")
        #     print(f"Error: {str(e)}")


In [ ]:
identify_latent_needs()

In [157]:
def generate_final_report():
    print("\n=== Generating Final Report ===")
    
    payload = {
        "interviews": interviews,
        "questions": questions
    }
    
    try:
        response = client.post("/create_report", json=payload)
        if response.status_code == 200:
            report_data = response.json()
            print("\nFinal Report:")
            print(report_data)
            return report_data
        else:
            print("\nError generating report")
            print(f"Status code: {response.status_code}")
            print(f"Response: {response.text}")
    except Exception as e:
        print("\nException during report generation")
        print(f"Error: {str(e)}")

def save_results():
    print("\n=== Saving Results ===")
    results = {
        "product": product,
        "model_steering": model_steering,
        "stakeholders": stakeholders,
        "interviews": interviews,
        # "latent_needs": latent_needs,
        "report": final_report,
        "questions": questions
    }
    
    # 使用时间戳作为文件名的一部分，确保唯一性
    from datetime import datetime
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"Elicitron_results_{timestamp}.json"
    
    import json
    with open(filename, 'w') as f:
        json.dump(results, f, indent=2)
    print(f"\nResults saved to {filename}")

# 执行整个流程
final_report = generate_final_report()
save_results()

DEBUG:asyncio:Using selector: KqueueSelector
DEBUG:backend:Interview 1: {'id': '92336490-f4e6-46c6-b481-d8f77ebd9279', 'stakeholder_type': 'Adaptive Clothing Enthusiast', 'personality_description': 'The user is highly adaptive, fashion-forward, and technology-savvy. They are enthusiastic about innovations in the clothing industry, especially those that promote inclusivity and adaptability. They are conscious of sustainability and appreciate garments that offer practicality without compromising style. They value personal expression through clothing and have a keen eye for detail.', 'product_usage_experience': "Step 1:\n- Action: I began by exploring the garment's adjustable features, excited to see how the size-changing technology works. I adjusted the garment to its smallest possible size using the tabs and buttons designed for flexibility.\n- Observation: The transition was smooth, and the material maintained its style without any awkward bunching or wrinkles. The buttons were intuiti


=== Generating Final Report ===


DEBUG:httpcore.http11:receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Thu, 24 Apr 2025 16:10:40 GMT'), (b'Content-Type', b'application/json'), (b'Transfer-Encoding', b'chunked'), (b'Connection', b'keep-alive'), (b'access-control-expose-headers', b'X-Request-ID'), (b'openai-organization', b'codesign-lab'), (b'openai-processing-ms', b'7392'), (b'openai-version', b'2020-10-01'), (b'x-ratelimit-limit-requests', b'10000'), (b'x-ratelimit-limit-tokens', b'30000000'), (b'x-ratelimit-remaining-requests', b'9999'), (b'x-ratelimit-remaining-tokens', b'29999700'), (b'x-ratelimit-reset-requests', b'6ms'), (b'x-ratelimit-reset-tokens', b'0s'), (b'x-request-id', b'req_56b7efe7ac6a1b1c35619d5de2dae983'), (b'strict-transport-security', b'max-age=31536000; includeSubDomains; preload'), (b'cf-cache-status', b'DYNAMIC'), (b'X-Content-Type-Options', b'nosniff'), (b'Server', b'cloudflare'), (b'CF-RAY', b'9356df16aeb4fae3-SJC'), (b'Content-Encoding', b'gzip'), (b'alt-sv


Final Report:
**Report on Key Insights and Requirements from Interview on Adjustable Garments**

**Introduction:**
This report synthesizes insights from an interview with an adaptive clothing enthusiast, focusing on the challenges faced with current garments, the potential benefits of adjustable clothing, and the desired features in such garments. The interviewee is characterized by a strong interest in fashion innovation, inclusivity, and sustainability, with a preference for garments that blend practicality and style.

**Challenges with Current Garments:**
The interviewee highlighted several challenges with existing clothing options. Primarily, they noted a lack of adaptability in garments, which often fails to accommodate diverse body types and personal style preferences. This rigidity can limit personal expression and comfort. Additionally, there is a concern about the environmental impact of traditional clothing production, with a need for more sustainable practices.

**Benefits 

In [174]:
def extract_and_save_customer_needs():
    print("\n=== Extracting Customer Needs ===")
    
    if not final_report:
        print("Error: No final report available")
        return
    
    payload = {
        "product": product,
        "report": final_report
    }
    
    try:
        response = client.post("extract_customer_needs", json=payload)
        print(response.text)
        if response.status_code == 200:
            needs_data = response.json()
            print("\nExtracted Customer Needs:")
            for i, need in enumerate(needs_data['customer_needs'], 1):
                print(f"{i}. {need}")
            
            # 更新结果字典，添加标准化的客户需求
            results = needs_data
            
            # 保存更新后的结果
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = f"results_with_needs_{timestamp}.json"
            
            with open(filename, 'w') as f:
                json.dump(results, f, indent=2)
            print(f"\nResults saved to {filename}")
            
            return needs_data
        else:
            print("\nError extracting customer needs")
            print(f"Status code: {response.status_code}")
            print(f"Response: {response.text}")
    except Exception as e:
        print("\nException during customer needs extraction")
        print(f"Error: {str(e)}")

In [ ]:
extract_and_save_customer_needs()

In [171]:
# 在 caller.ipynb 中添加以下代码

import importlib
import backend
importlib.reload(backend)  # 重新加载模块
from backend import app
from fastapi.testclient import TestClient

# 重新初始化 TestClient
client = TestClient(app)

# 首先验证端点是否正确注册
print("Available routes:")
for route in app.routes:
    print(f"- {route.path} [{route.methods}]")

# 测试 extract_customer_needs 端点
def test_extract_customer_needs():
    payload = {
        "product": "size changing and adjustable garment",
        "report": """
        The interviews revealed several key insights about the adjustable garment. Users emphasized the need for intuitive size adjustment mechanisms and comfortable materials that adapt to movement. Sustainability was highlighted as important, with users expressing interest in eco-friendly materials and durability. The garment should maintain its adjustability while being easy to clean and maintain.
        """
    }
    
    response = client.post("/extract_customer_needs", json=payload)
    print("\nResponse Status:", response.status_code)
    print("Response Content:", response.text)
    
    if response.status_code != 200:
        print(f"Error: {response.status_code}")
        print(f"Response: {response.text}")
    else:
        print("Success!")
        print(response.json())

# 运行测试
test_extract_customer_needs()

DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=True http2=False
DEBUG:httpx:load_verify_locations cafile='/Users/lindadexiaoaojiao/miniconda3/envs/agent/lib/python3.12/site-packages/certifi/cacert.pem'
DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=True http2=False
DEBUG:httpx:load_verify_locations cafile='/Users/lindadexiaoaojiao/miniconda3/envs/agent/lib/python3.12/site-packages/certifi/cacert.pem'
DEBUG:asyncio:Using selector: KqueueSelector
DEBUG:backend:Extracting customer needs for size changing and adjustable garment
DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'user', 'content': '\nBased on the following report about size changing and adjustable garment, extract and standardize the customer needs.\n\nReport:\n\n        The interviews revealed several key insights about the adjustable garment. Users emphasized the need for intuitive size adjustment mechanis

Available routes:
- /openapi.json [{'HEAD', 'GET'}]
- /docs [{'HEAD', 'GET'}]
- /docs/oauth2-redirect [{'HEAD', 'GET'}]
- /redoc [{'HEAD', 'GET'}]
- /create_stakeholder_parallel [{'POST'}]
- /create_stakeholder_serial [{'POST'}]
- /conduct_interview [{'POST'}]
- /filter_top_diverse [{'POST'}]
- /create_report [{'POST'}]
- /latent_need_identification [{'POST'}]
- /extract_customer_needs [{'POST'}]
=========This is getting called=========


DEBUG:httpcore.http11:receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Thu, 24 Apr 2025 16:13:03 GMT'), (b'Content-Type', b'application/json'), (b'Transfer-Encoding', b'chunked'), (b'Connection', b'keep-alive'), (b'access-control-expose-headers', b'X-Request-ID'), (b'openai-organization', b'codesign-lab'), (b'openai-processing-ms', b'2149'), (b'openai-version', b'2020-10-01'), (b'x-ratelimit-limit-requests', b'10000'), (b'x-ratelimit-limit-tokens', b'30000000'), (b'x-ratelimit-remaining-requests', b'9999'), (b'x-ratelimit-remaining-tokens', b'29999682'), (b'x-ratelimit-reset-requests', b'6ms'), (b'x-ratelimit-reset-tokens', b'0s'), (b'x-request-id', b'req_0e31bb1d85337804ba5c74cf789b6cfe'), (b'strict-transport-security', b'max-age=31536000; includeSubDomains; preload'), (b'cf-cache-status', b'DYNAMIC'), (b'Set-Cookie', b'__cf_bm=unuj6JchdYhvk_.2oWDXuP8a1Ddnf_hYtRejDc3wPPo-1745511183-1.0.1.1-YcKlM2VFvcl8VWP6NGzZFFj.Nv6trffQbqhhi_Vh3OeXd94Ry2GL.J8.bZa


Response Status: 200
Response Content: {"product":"size changing and adjustable garment","customer_needs":["The size changing and adjustable garment allows intuitive size adjustment.","The size changing and adjustable garment provides comfort with materials that adapt to movement.","The size changing and adjustable garment uses eco-friendly materials.","The size changing and adjustable garment maintains durability.","The size changing and adjustable garment remains easy to clean.","The size changing and adjustable garment is easy to maintain."]}
Success!
{'product': 'size changing and adjustable garment', 'customer_needs': ['The size changing and adjustable garment allows intuitive size adjustment.', 'The size changing and adjustable garment provides comfort with materials that adapt to movement.', 'The size changing and adjustable garment uses eco-friendly materials.', 'The size changing and adjustable garment maintains durability.', 'The size changing and adjustable garment remains 

In [32]:
from fastapi.testclient import TestClient
from backend import app  # Import your FastAPI app

# Initialize TestClient
client = TestClient(app)

def test_create_and_interview_stakeholder():
    # Step 1: Create stakeholders using /create_stakeholder_serial
    create_payload = {
        "design_context": "size changing and adjustable garment",
        "model_steering": "Focus on adaptive wearability",
        "number_of_stakeholders": 3
    }

    create_response = client.post("/create_stakeholder_serial", json=create_payload)
    assert create_response.status_code == 200, f"Error in stakeholder creation: {create_response.status_code}, {create_response.text}"

    # Extract stakeholders from the response
    created_stakeholders = create_response.json().get("stakeholders", [])
    assert len(created_stakeholders) > 0, "No stakeholders created"

    # Pick the first stakeholder for the interview
    stakeholder = created_stakeholders[0]
    print("Selected Stakeholder for Interview:", stakeholder)

    # Ensure the stakeholder has an 'id' field
    if 'id' not in stakeholder:
        stakeholder['id'] = "001"  # Assign a default id if missing

    # Step 2: Conduct interview using /conduct_interview
    interview_payload = {
        "stakeholder": stakeholder,
        "design_context": "size changing and adjustable garment",
        "questions": [
            "What features do you look for in adjustable clothing?",
            "How important is sustainability in your fashion choices?"
        ]
    }

    interview_response = client.post("/conduct_interview", json=interview_payload)
    assert interview_response.status_code == 200, f"Error in conducting interview: {interview_response.status_code}, {interview_response.text}"

    # Output the interview result
    interview_data = interview_response.json()
    print("Interview Result:", interview_data)

    # Optional: Validate interview content
    assert "stakeholder_type" in interview_data or "interview_summary" in interview_data, "Interview response missing expected fields"


test_create_and_interview_stakeholder()


DEBUG:asyncio:Using selector: KqueueSelector
DEBUG:backend:Creating 3 users.
DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'user', 'content': "You're a user creator for a new design of size changing and adjustable garment. You MUST consider the following: Focus on adaptive wearability. The users must be diverse consumers of the product, whom you will interview afterwards to identify user needs. Please create a list of 3 users."}], 'model': 'gpt-4o', 'function_call': {'name': 'StakeholderList'}, 'functions': [{'name': 'StakeholderList', 'description': 'Correctly extracted `StakeholderList` with all the required parameters with correct types', 'parameters': {'$defs': {'StakeholderBase': {'properties': {'chain_of_thought': {'description': '\nExplain why this stakeholder is important to the design process.\n', 'title': 'Chain Of Thought', 'type': 'string'}, 'stakeholder_type': {'description': 'Ty

Selected Stakeholder for Interview: {'chain_of_thought': 'This stakeholder represents everyday consumers who prioritize versatility and practicality in their clothing choices. Understanding their needs will help ensure the product is functional for daily wear and can adapt to different body sizes and lifestyles.', 'stakeholder_type': 'Casual Wear Enthusiast', 'personality_description': 'Practical, fashion-forward, values comfort and ease of use, enjoys clothing that can transition between different settings, busy lifestyle requiring adaptive apparel.'}


DEBUG:httpcore.http11:receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Thu, 24 Apr 2025 13:44:26 GMT'), (b'Content-Type', b'application/json'), (b'Transfer-Encoding', b'chunked'), (b'Connection', b'keep-alive'), (b'access-control-expose-headers', b'X-Request-ID'), (b'openai-organization', b'codesign-lab'), (b'openai-processing-ms', b'28861'), (b'openai-version', b'2020-10-01'), (b'x-ratelimit-limit-requests', b'10000'), (b'x-ratelimit-limit-tokens', b'30000000'), (b'x-ratelimit-remaining-requests', b'9999'), (b'x-ratelimit-remaining-tokens', b'29999916'), (b'x-ratelimit-reset-requests', b'6ms'), (b'x-ratelimit-reset-tokens', b'0s'), (b'x-request-id', b'req_7d06e4823aceb46cde041864bc4baf04'), (b'strict-transport-security', b'max-age=31536000; includeSubDomains; preload'), (b'cf-cache-status', b'DYNAMIC'), (b'X-Content-Type-Options', b'nosniff'), (b'Server', b'cloudflare'), (b'CF-RAY', b'935608572bd4cf26-SJC'), (b'Content-Encoding', b'gzip'), (b'alt-s

Interview Result: {'id': '001', 'stakeholder_type': 'Casual Wear Enthusiast', 'personality_description': 'Practical, fashion-forward, values comfort and ease of use, enjoys clothing that can transition between different settings, busy lifestyle requiring adaptive apparel.', 'product_usage_experience': "Step 1:\n- Action: On receiving the garment, I explored the material and design features by trying it on.\n- Observation: The fabric felt soft and breathable, perfect for a casual wear enthusiast who values comfort. The design appeared stylish and could easily transition between casual and smart-casual settings.\n- Challenge: Initially, understanding the adjustment mechanisms was slightly challenging as it required careful inspection.\n\nStep 2:\n- Action: I tested the size adjustment feature by resizing the garment to fit different activities.\n- Observation: The garment could be easily adjusted using concealed straps and buttons, allowing it to either tighten for a more fitted look or 